In [19]:
import pandas
import matplotlib.pyplot as plt
import matplotlib
import pickle
import wntr
import numpy as np
import pandas as pd

import os
import sys
sys.path.append('../Code/')

from testWN import testWN as twm

from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration, SpectralClustering
from sklearn.decomposition import PCA

matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['svg.fonttype'] = 'none'

In [2]:
%matplotlib qt

# Get Network Information

In [3]:
inp_file = '../Code/c-town_true_network_simplified_controls.inp'
ctown = twm(inp_file)
node_names_all = ctown.wn.node_name_list
node_names = ctown.getNodeName()
link_names = ctown.getLinkName()

n_junctions = len(node_names[2])
n_nodes = len(node_names_all)

pump_df = pd.DataFrame(np.ones(len(link_names[0])).reshape(1,-1), columns=link_names[0])
valves_df = pd.DataFrame(np.ones(len(link_names[2])).reshape(1,-1), columns=link_names[2])

/home/ffiedler/miniconda3/envs/py37_WNTR/lib/python3.7/site-packages/wntr/epanet/io.py:2088: UserWarning:

Not all curves were used in "../Code/c-town_true_network_simplified_controls.inp"; added with type None, units conversion left to user



# Get Connectivity Matrix

Infos about connectivity matrix [here](https://transportgeography.org/?page_id=6969).

In [4]:
A = np.zeros((n_junctions,n_junctions))
links = ctown.wn.links.todict()
for key in links.keys():
    i = np.where([np.array(node_names[2]) == links[key]['start_node_name']])[1]
    j = np.where([np.array(node_names[2]) == links[key]['end_node_name']])[1]
    
    A[i,j] = 1
    A[j,i] = 1

In [5]:
plt.figure()
plt.spy(A)

# Get Coordinates of nodes:

In [6]:
nodes = ctown.wn.nodes.todict()

In [7]:
coords_arr = np.array([nodes[key]['coordinates'] for key in nodes.keys()])
coords = pd.DataFrame(coords_arr, index=node_names_all)

# Clustering

## Prepocess Data:

### with random control input:

In [9]:
file_rc = '/home/ffiedler/tubCloud/Shared/WDN_SurrogateModels/_RESULTS/150sim_1hourSampling/results_sim_59.pkl'

with open(file_rc, 'rb') as f:
    res_rc=pickle.load(f)

### With MPC control:

In [8]:
file_mpc = '/home/ffiedler/tubCloud/Shared/WDN_SurrogateModels/_RESULTS/MPC/001_economic/006_mod_007_results_sim_time.pkl'

with open(file_mpc, 'rb') as f:
    res_mpc=pickle.load(f)

# Load cluster information:

In [12]:
cluster_labels = pd.read_json('cluster_labels_only_mpc_02.json')
pressure_factor = pd.read_json('pressure_factor_only_mpc_02.json')
n_clusters = 30

In [29]:
cmap = matplotlib.cm.get_cmap('tab20')

fig_1, ax_1 = plt.subplots(1,1, figsize=(5,4), sharex=True, sharey=True)

press = res_mpc.node['pressure'][node_names[2]]
press_med = press.mean()

# # Highlight pumps
# wntr.graphics.plot_network(ctown.wn, link_attribute=pump_df.loc[0],node_size=1, link_width=6, 
#                            add_colorbar=False, title='Pumps', ax=ax_1[0])
# # Highlight valves
# wntr.graphics.plot_network(ctown.wn, link_attribute=valves_df.loc[0],node_size=1, link_width=6, 
#                            add_colorbar=False, title='Valves', ax=ax_1[0])

lines = wntr.graphics.plot_network(ctown.wn, node_attribute=cluster_labels.loc['pressure_cluster'],
                                   node_size=50, node_labels=False, node_cmap='tab20',
                                   add_colorbar=False, title='junction clustering', ax=ax_1) 

# wntr.graphics.plot_network(ctown.wn, node_attribute=press_med[node_names[2]],node_size=50, 
#                            node_cmap='CMRmap', title='mean pressure', ax=ax_1[1])  

# mesh = ax_1[1].collections[1]
# mesh.colorbar.set_label('pressure [m]')

if True:
    for i in range(n_clusters):
        cluster_mean = coords.loc[node_names[2]][cluster_labels.loc['pressure_cluster']==i].median()-100
        ax_1.text(*cluster_mean.tolist(), str(i), ha="center", va="center",
                     bbox=dict(boxstyle="round", ec='k', fc=cmap(i/n_clusters), alpha=0.7))
fig_1.tight_layout(pad=0.1)



### Evaluation

In [37]:
press_junc_mpc = res_mpc.node['pressure'][node_names[2]]
press_junc_mpc.index = press_junc_mpc.index/3600

pressure_factor = press_junc_mpc.abs().mean()

n_press_junc_mpc = press_junc_mpc / pressure_factor.to_numpy()
# Calculate cluster mean and standard deviation
jun_cl_press_group_mpc = press_junc_mpc.groupby(cluster_labels.loc['pressure_cluster'], axis=1)
jun_cl_n_press_group_mpc = n_press_junc_mpc.groupby(cluster_labels.loc['pressure_cluster'], axis=1)
jun_cl_n_press_mean_mpc = jun_cl_n_press_group_mpc.mean()
jun_cl_n_press_std_mpc = jun_cl_n_press_group_mpc.std()

In [38]:
fig, ax = plt.subplots(2,2,figsize=(9,5), sharex=True, sharey='row')


cluster_1=4
cluster_2=0

ax[0,0].plot(jun_cl_press_group_mpc.get_group(cluster_1),color='k', linewidth=0.2, alpha=0.5)

ax[1,0].plot(jun_cl_n_press_group_mpc.get_group(cluster_1),color='k', linewidth=0.2, alpha=0.5)
ax[1,0].plot(jun_cl_n_press_mean_mpc[[cluster_1]], label='cluster mean', linewidth=2)


ax[0,1].plot(jun_cl_press_group_mpc.get_group(cluster_2),color='k', linewidth=0.2, alpha=0.5)
ax[1,1].plot(jun_cl_n_press_group_mpc.get_group(cluster_2),color='k', linewidth=0.2, alpha=0.5)
ax[1,1].plot(jun_cl_n_press_mean_mpc[[cluster_2]], label='cluster mean', linewidth=2)


ax[0,0].set_title('MPC \n cluster #{}  with n={} elements'.format(cluster_1, jun_cl_press_group_mpc.get_group(cluster_1).shape[1]))
ax[0,1].set_title('Random control input \n cluster #{}  with n={} elements'.format(cluster_2, jun_cl_press_group_mpc.get_group(cluster_1).shape[1]))
ax[1,0].set_xlabel('time [h]')
ax[1,1].set_xlabel('time [h]')
ax[0,0].set_ylabel('pressure [m]')
ax[1,0].set_ylabel('norm. \n pressure [-]')
#ax[1,0].set_ylim(0,1)
#ax[0,0].set_ylim(40,200)
#ax[0,0].set_xlim(0,100)

# ax_2.fill_between(jun_cl_press_mean.index,
#                   (jun_cl_press_mean[[cluster_i]]-3*jun_cl_press_std[[cluster_i]]).to_numpy().flatten(),
#                   (jun_cl_press_mean[[cluster_i]]+3*jun_cl_press_std[[cluster_i]]).to_numpy().flatten(), 
#                   alpha=0.5, label='$\pm 3\sigma$')
# ax_2.plot(jun_cl_press_group.get_group(cluster_i),color='k', linewidth=0.2, alpha=0.5)
# ax_2.plot(jun_cl_press_mean[[cluster_i]], label='cluster mean')
# ax_2.plot([0,0],[0,0], color='k', linewidth=0.2, alpha=0.5, label='individual nodes')

# ax_2.legend()
# ax_2.set_title('Normalized pressure for cluster #{}  with n={} elements'.format(cluster_i, jun_cl_press_group.get_group(cluster_i).shape[1]))
# ax_2.set_xlabel('time [s]')
# ax_2.set_ylabel('Pressure')

fig.align_ylabels()
fig.tight_layout(pad=0.2, w_pad=0.2, h_pad=0)